* Follows idea from [this paper](https://arxiv.org/pdf/2403.01206) - take same division algorithm using different adders as building blocks.
* Dividers (Restoring and Non-Restoring) from [this paper](https://arxiv.org/pdf/1809.09732).

In [1]:
import qsharp
import json
from diskcache import Cache
import re_utils

re_utils.DEBUG = True

cache = Cache("~/quant-arith-cache/re-dividers")
qsharp.init(project_root="../")

@cache.memoize()
def estimate_resources_divide(op, n):
    if ";" in op:
        divider_type, adder = op.split(";")
        if divider_type == "Restoring":
            est = qsharp.estimate(f"QuantumArithmetic.TMVH2019Test.RunForRE_Restoring({n},{adder})")
        elif divider_type == "NonRestoring":
            est = qsharp.estimate(f"QuantumArithmetic.TMVH2019Test.RunForRE_NonRestoring({n},{adder})")
        else:
            raise ValueError("Unknown divider_type")
    elif op == "AKBF2011.Divide_Restoring":
        est = qsharp.estimate(f"QuantumArithmetic.AKBF2011Test.RunForRE_Divide_Restoring({n})")
    else:
        raise ValueError("Unknown op")
    return json.dumps(est)

ops_and_max_n = [
  ("NonRestoring;Std.Arithmetic.RippleCarryCGIncByLE", "NonRestoring+Gidney", 2**14),
  ("NonRestoring;Std.Arithmetic.RippleCarryTTKIncByLE", "NonRestoring+TTK", 2**14),
  ("NonRestoring;QuantumArithmetic.CDKM2004.Add", "NonRestoring+CDKM", 2**12),
  ("NonRestoring;QuantumArithmetic.JHHA2016.Add_Mod2N", "NonRestoring+JHHA", 2**12),
  ("Restoring;Std.Arithmetic.RippleCarryCGIncByLE", "Restoring+Gidney", 2**12),
  ("Restoring;Std.Arithmetic.RippleCarryTTKIncByLE", "Restoring+TTK", 2**14),
  ("Restoring;QuantumArithmetic.CDKM2004.Add", "Restoring+CDKM", 2**12),
  ("Restoring;QuantumArithmetic.JHHA2016.Add_Mod2N", "Restoring+JHHA", 2**12),
  ("AKBF2011.Divide_Restoring", "AKBF", 2**12),
]

re_utils.run_re_experiments(
    ops_and_max_n, 
    estimate_resources_divide,
    title='(e) Division')

Charts saved to img/e_division.png


In [2]:
ops_and_max_n += [
  ("NonRestoring;QuantumArithmetic.DKRS2004.Add", "NonRestoring+DKRS", 2**10),
  ("NonRestoring;Std.Arithmetic.FourierTDIncByLE", "NonRestoring+QFT", 2**7),
  ("Restoring;QuantumArithmetic.DKRS2004.Add", "Restoring+DKRS", 2**10),
  ("Restoring;Std.Arithmetic.FourierTDIncByLE", "Restoring+QFT", 2**7),
]

re_utils.show_re_table(
    ops_and_max_n, 
    estimate_resources_divide, 2**7)

n=128


,Algorithm,Logical qubits,Physical qubits,Logical depth,"Runtime, seconds"
0,NonRestoring+Gidney,1087,1018814,65909,0.500908
1,NonRestoring+TTK,823,900206,98673,0.749915
2,NonRestoring+CDKM,827,903094,100793,0.766027
3,NonRestoring+JHHA,827,903094,100844,0.766414
4,Restoring+Gidney,1089,1176498,179200,1.505280
5,Restoring+TTK,825,1015650,244224,2.051482
6,Restoring+CDKM,829,1019178,566912,4.762061
7,Restoring+JHHA,829,1001178,425984,3.578266
8,AKBF,1880,1928160,425728,3.576115
9,NonRestoring+DKRS,1320,1434240,370439,3.111688


In [3]:
re_utils.trendline_analysis(
    ops_and_max_n, 
    estimate_resources_divide)

,Algorithm,Logical qubits,Physical qubits,Logical depth,"Runtime, seconds"
0,NonRestoring+Gidney,8.8739e+00 * n^0.9890,3.6059e+03 * n^1.1534,4.0352e+00 * n^1.9990,1.9073e-05 * n^2.1029
1,NonRestoring+TTK,6.7594e+00 * n^0.9874,3.7108e+03 * n^1.1180,6.0349e+00 * n^1.9993,2.8986e-05 * n^2.1018
2,NonRestoring+CDKM,6.9890e+00 * n^0.9823,4.1844e+03 * n^1.0980,6.3248e+00 * n^1.9932,2.8789e-05 * n^2.1045
3,NonRestoring+JHHA,6.9890e+00 * n^0.9823,4.1844e+03 * n^1.0980,6.3308e+00 * n^1.9931,2.8817e-05 * n^2.1043
4,Restoring+Gidney,9.0901e+00 * n^0.9852,4.4416e+03 * n^1.1336,1.0869e+01 * n^2.0015,5.2291e-05 * n^2.1091
5,Restoring+TTK,6.7848e+00 * n^0.9869,4.0443e+03 * n^1.1151,1.4856e+01 * n^2.0011,7.4083e-05 * n^2.1028
6,Restoring+CDKM,7.0316e+00 * n^0.9815,5.6243e+03 * n^1.0730,3.4169e+01 * n^2.0031,1.7750e-04 * n^2.1040
7,Restoring+JHHA,7.0316e+00 * n^0.9815,5.1646e+03 * n^1.0822,2.6000e+01 * n^2.0000,1.3168e-04 * n^2.1031
8,AKBF,1.5460e+01 * n^0.9885,7.6228e+03 * n^1.1467,2.5967e+01 * n^2.0002,1.3595e-04 * n^2.1026
9,NonRestoring+DKRS,1.0752e+01 * n^0.9918,6.1034e+03 * n^1.1251,2.0476e+01 * n^2.0225,1.0121e-04 * n^2.1321
